In [2]:
import numpy as np
import pandas as pd
import datetime
from dateutil.parser import parse
import requests, re, time
import pandas_datareader
import pickle
import math

import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib.lines import Line2D
import matplotlib.patches as mpatches
import seaborn as sns 

import statsmodels.api as sm
import json
from pandas.io.json import json_normalize
from bs4 import BeautifulSoup
from selenium import webdriver
import time

import geopy.distance

# The Impact of Metro-Construction: An Event Study

In the following, I seek to find the effect on the price of Copenhagen apartments from the construction of a nearby metro. 

First, I scrape data of all sold condos in Copenhagen since 1992 using Boliga.dk

Second, I use Google's Geocoding-API to convert the found addresses to coordinates. This will help me in step three.

Third, I scrape Wikipedia for information on Copenhagen Metro stations.

Fourth, I want to find the cloest metro to each apartment, and estimate the distance between the two. This is found to be most easily done by finding coordinates to each apartment and metro, calculate the distance between each apartment and its closest metro and therefrom infer a distance. As the found distance is in the form of a straight line, I estimate the walking distance by using the Pythagorean Theorem. From this, I can infer an estimated walking time.

Fifth, I must take relevant factors into account in an econometric analysis. (Possibly using ML for slope estimation in regression?)

This enables me to isolate the effect on the price of a condo from construction of a nearby metro station.

(The Cityring-law was put into place at 06-06-2007, and the construction work started in 2009. It opened in 2019. Due to the initialization of construction around the peak of the financial and economic crisis, unbiased estimation of the Cityrings effect is very difficult.)

# 1) Scraping sales-data from Boliga.dk

## 1.1) Scraping

### Defining aiding functions:

In [ ]:
path2gecko = '/Users/holger/Documents/Python/Harmsen_Repo/Metro-Study/Gecko/geckodriver'

def Cookie_Clicker(i):
        #Accepting cookies:
    if i==True:
        cookies = browser.find_element_by_xpath('//*[@id="coiAccept"]')
        cookies.click()
        
def Next_Page(i):
    if i==True:
        next_button = browser.find_element_by_link_text('Næste')
        next_button.click()

### Defining the scraping-function:

In [ ]:
browser = webdriver.Firefox(executable_path=path2gecko) # start the browser with a path to the geckodriver.

def Scraper(pages,url):
    url_base = url
    
    browser.get(url_base)

    #Waiting for cookies-pop-up:
    time.sleep(10)

    #Accepting cookies:
    Cookie_Clicker(True)

    #Wait for load:
    time.sleep(5)

    #Pull table
    df = pd.read_html(browser.page_source)[0]
    time.sleep(2)

    #Looping over number of wanted pages:
    for i in range(0,pages-1):
        Next_Page(True)
        time.sleep(2)
        df_i = pd.read_html(browser.page_source)[0]
        df = pd.concat([df,df_i],ignore_index=True)
        
    return df

### Calling scraper:

In [ ]:
# Scraping Copenhagen
url_broaderCPH = 'https://www.boliga.dk/salg/resultater?propertyType=3&salesDateMin=1992&street=&municipality=101&page=1&sort=date-d'

df_broaderCPH = Scraper(1110,url_broaderCPH)

In [ ]:
# Scraping Frederiksberg

url_FRB = 'https://www.boliga.dk/salg/resultater?propertyType=3&salesDateMin=1992&street=&municipality=147&page=1&sort=date-d'

df_FRB = Scraper(247,url_FRB)


In [ ]:
df_Total = pd.concat([df_broaderCPH,df_FRB],ignore_index=True)

## 1.2) Working with the data:

In [ ]:
Apartments = pd.DataFrame()
Apartments['Address'] = df_Total['Adresse']

dates = [i[10:] for i in df_Total['Salgsdato']]
Apartments['Date_sold'] = [datetime.datetime(year=int(i[6:]),month=int(i[3:5]),day=int(i[:2]),)
    for i in dates
]

Apartments['Price'] = [int(i[8:].replace('.',''))
    for i in df_Total['Købesum']
]

Apartments['Price_sq_m'] = [int(i[9:].replace('.',''))
    for i in df_Total['Kr. / m²']
]

Apartments['Rooms'] = [int(i[9:])
    for i in df_Total['Værelser']
]

Apartments.to_pickle('/Users/holger/Documents/Python/Harmsen_Repo/Metro-Study/Pickles/Apartment Data 1.pkl')

In [44]:
Apartments = pickle.load(open('/Users/holger/Documents/Python/Harmsen_Repo/Metro-Study/Pickles/Apartment Data 1.pkl','rb'))
Apartments.head(n=3)

,Address,Date_sold,Price,Price_sq_m_1000,Rooms
0,"Bogholder Allé 15, 3. tv, 2720 Vanløse",2020-07-16,2340000,33.429,3
1,"Theklavej 49, 2. 79, 2400 København NV",2020-07-16,1550000,38.750,1
2,"Rådmandsgade 48A, 2. th, 2200 København N",2020-07-16,816000,14.836,2


A new column is added, using RegEx to remove all text between the commas. This substring expresses which door in the given apartment complex the address belongs to, and is thus irrelevant for the distance to nearest Metro. This is due to a later issue with the API not being able to recognize the address otherwise.

In [46]:
Apartments['Address transformed'] = [re.sub(',[^>]+,', '',i)
for i in Apartments['Address']
    ]
Apartments.head(n=3)

,Address,Date_sold,Price,Price_sq_m_1000,Rooms,Address transformed
0,"Bogholder Allé 15, 3. tv, 2720 Vanløse",2020-07-16,2340000,33.429,3,Bogholder Allé 15 2720 Vanløse
1,"Theklavej 49, 2. 79, 2400 København NV",2020-07-16,1550000,38.750,1,Theklavej 49 2400 København NV
2,"Rådmandsgade 48A, 2. th, 2200 København N",2020-07-16,816000,14.836,2,Rådmandsgade 48A 2200 København N


### Sorting data by address and date sold:

In [47]:
data_sorted = Apartments.sort_values(['Address','Date_sold'])

# Problem with some pgaes being scraped more than once - deleting duplicate rows:
data_sorted.drop_duplicates(inplace=True)
data_sorted = data_sorted.sort_values(['Date_sold'])
data_sorted['Price_mio'] = data_sorted['Price']/10**6
data_sorted = data_sorted.reset_index()

# Sorting and creating daily averages and moving average for graphing:
dailyavg = data_sorted.groupby('Date_sold').agg(
    Dailyavg_mio = ('Price_mio','mean'),
    Dailyavg_sq = ('Price_sq_m_1000','mean')
)
dailyavg['Day'] = dailyavg.index.to_list()
dailyavg['MovAvg'] = dailyavg['Dailyavg_sq'].rolling(window=500,center=True).mean()

In [48]:
# The apartment data now looks like this:
Apartments = data_sorted
Apartments.head(n=3)

,index,Address,Date_sold,Price,Price_sq_m_1000,Rooms,Address transformed,Price_mio
0,67848,"Finsensvej 47B, 4. 410, 2000 Frederiksberg",1992-01-02,335000,5.583,1,Finsensvej 47B 2000 Frederiksberg,0.335
1,55498,"Øresundsvej 108, 1. tv, 2300 København S",1992-01-03,351000,5.754,2,Øresundsvej 108 2300 København S,0.351
2,67847,"Frederik VIs Alle 6, 3. tv, 2000 Frederiksberg",1992-01-03,740000,7.048,4,Frederik VIs Alle 6 2000 Frederiksberg,0.740


In [49]:
# The grouped mean- and moving average-data looks like this (MovAvg has NaN-values in beginning and ending of series):
dailyavg.head(n=3)

,Dailyavg_mio,Dailyavg_sq,Day,MovAvg
Date_sold,,,,
1992-01-02,0.3350,5.583,1992-01-02,NaN
1992-01-03,0.5455,6.401,1992-01-03,NaN
1992-01-04,0.2500,5.000,1992-01-04,NaN


### Splitting the apartments by sub-areas within Copenhagen:

In [37]:
Apartments['Area'] = 'NaN'
for i in range(0,len(Apartments)):
    if Apartments['Address'][i][-1] == 'N':
        Apartments['Area'][i] = 'N'
        # Solving issue with differentiating between NV and V:
    elif Apartments['Address'][i][-2:] == 'NV':
        if Apartments['Address'][i][-2] == ' V':
            Apartments['Area'][i] = 'V'
        else: 
            Apartments['Area'][i] = 'NV'
    elif Apartments['Address'][i][-1] == 'Ø':
        Apartments['Area'][i] = 'Ø'
    elif Apartments['Address'][i][-1] == 'K':
        Apartments['Area'][i] = 'K'
    elif Apartments['Address'][i][-1] == 'S':
        Apartments['Area'][i] = 'S'
    elif Apartments['Address'][i][-5:] == 'Valby':
        Apartments['Area'][i] = 'Valby'
    elif Apartments['Address'][i][-13:] == 'Frederiksberg':
        Apartments['Area'][i] = 'FRB'
    elif Apartments['Address'][i][-1:] == 'C':
        Apartments['Area'][i] = 'FRB'

In [39]:
Apartments.to_pickle('/Users/holger/Documents/Python/Harmsen_Repo/Metro-Study/Pickles/Apartment Data 2.pkl')

# 2) Geocoding: Address to coordinates

## 2.1) Defining the Geocoding-function, finding coordinates from a given address:

Defining a function which returns the coordinates for a given address, using the Google Geocode API. I am using a free voucher of DKK 2,000. I need to pull coordinates for around 45,100 adresses, this will spend around DKK 1,400.

In [10]:
geo_url = 'https://maps.googleapis.com/maps/api/geocode/json'
def Geocode_Google(address):
    spec_params = {'key' : , #REMOVED
          'address' : address,
    }
    d = requests.get(geo_url, params=spec_params) # submit GET request based on url and headers
    json_data = d.json()
    subdata = json_data['results']
    return (subdata[0]['geometry']['location']['lat'],subdata[0]['geometry']['location']['lng'])

## 2.2) Finding Coordinates to each apartment:

Below, I use the Geocode-function to pull coordinates for all apartments in the dataset. This process took around 8 hours, so I split it up into stages, saving to pickle for each pull in intervals of 5000 pulls, after which I concatenate to a single dataframe and add this column to the original dataset for each apartment. Below, this process has been simplified for easier readability.

In [ ]:
Apartments['Coordinates'] = [Geocode_Google(i) for i in Apartments['Address transformed']]

In [ ]:
Apartments.to_pickle('/Users/holger/Documents/Python/Harmsen_Repo/Metro-Study/Pickles/Apartment Data 3.pkl')

In [50]:
Apartments = pickle.load(open('/Users/holger/Documents/Python/Harmsen_Repo/Metro-Study/Pickles/Apartment Data 3.pkl','rb'))

In [51]:
Apartments.head(n=3)

,Address,Address transformed,Area,Coordinates,Date_sold,Price,Price_mio,Price_sq_m_1000,Rooms
0,"Stengade 52B, 1. 1, 2200 København N",Stengade 52B 2200 København N,N,"(55.687083, 12.554688)",1994-02-11,300710,0.300710,5.896,1
1,"Bryggerivej 10, 1. th, 2500 Valby",Bryggerivej 10 2500 Valby,Valby,"(55.660475, 12.518974)",1994-02-11,344101,0.344101,5.550,2
2,"Solvænget 5, 3. tv, 2100 København Ø",Solvænget 5 2100 København Ø,Ø,"(55.719198, 12.57804)",1994-02-11,1090300,1.090300,7.788,4


# 3) Scraping Metro-Data

By scraping danish Wikipedia-sites for Metro stations in Copenhagen, I can find Address, year built and other relevant information.

One problem is, that the information-tables on Wikipedia-pages do not contain the same number of rows and thus amount of information. This is resolved by creating a list of dictionaries, in which each element is a dictionary involving all information from a given table.
Selected information is drawn from here and appended to a finished DataFrame, containing all relevant Metro-information.

## 3.1) Scraping:

In [ ]:
browser = webdriver.Firefox(executable_path=path2gecko) # start the browser with a path to the geckodriver.

In [ ]:
def WikiScraper(no_metros):
    url_base='https://da.wikipedia.org/wiki/Stationer_p%C3%A5_K%C3%B8benhavns_Metro'
    browser.get(url_base)
    time.sleep(1)

    # Creating DataFrame and dictionary for information-storing:
    data = pd.DataFrame()
    dic_list = []

    for i in range(1,no_metros+1):
        link = browser.find_element_by_xpath(f'/html/body/div[3]/div[3]/div[5]/div/table[1]/tbody/tr[{i}]/td[1]/a')
        link.click()
        time.sleep(2)


        # READ INFORMATION
        # 1) Appending all information to DataFrame
        table = pd.read_html(browser.page_source)[0]
        data[f'Traits_{i}'] = table.iloc[:,0]
        data[f'{i}'] = table.iloc[:,1]

        # 2) Creating a dictionary with all information:
        dic = {}
        for j in range(0,len(data[f'Traits_{i}'])):
            dic[str(data[f'Traits_{i}'][j])] = str(data[f'{i}'][j])

        dic_list.append(dic)

        # RETURN 
        browser.get(url_base)
        time.sleep(2)


    return [data, dic_list]

In [ ]:
Metros = WikiScraper(44)

## 3.2) Working with the data:

Creating a DataFrame using the information from dictionaries:

In [ ]:
indexlist = ['Adresse','Jernbane','Åbningsdato']
MetroList = pd.DataFrame(index=indexlist)

for i in range(0,44):   
    specs = []
    for j in indexlist:
        try:
            specs.append(Metros[1][i][j])
        except:
            specs.append('NaN')
    # Indexing column names:
    MetroList[i] = specs
MetroList = MetroList.transpose()
MetroList.columns = ['Address','Railroad','Opening Date']

# Cleaning for '[1]' (from Wikipedia-references):
MetroList['Address'] = [i.replace('[1]','') for i in MetroList['Address']]
MetroList['Railroad'] = [i.replace('[1]','') for i in MetroList['Railroad']]
MetroList['Opening Date'] = [i.replace('[1]','') for i in MetroList['Opening Date']]

In [ ]:
MetroList.head(n=3)

In [ ]:
# Cleaning NaN-values:
MetroList = MetroList.drop(MetroList["Address"].loc[MetroList["Address"]=='NaN'].index)

In [ ]:
MetroList.to_pickle('/Users/holger/Documents/Python/Harmsen_Repo/Metro-Study/Pickles/Metrolist Base.pkl')

In [52]:
MetroList = pickle.load(open('/Users/holger/Documents/Python/Harmsen_Repo/Metro-Study/Pickles/Metrolist Base.pkl','rb'))

In [53]:
MetroList.head(n=3)

,Address,Railroad,Opening Date
0,Godthåbsvej 312000 Frederiksberg,Cityringen,29. september 2019
1,Italiensvej 72A2300 København S,Østamagerbanen,28. september 2007
2,Amagerbro Torv 12300 København S,Ørestadsbanen,19. oktober 2002


Seeing that the coordinates returned equals (39.78373,-100.445882) if there's a mistake (mostly, that the door and zip code are not separated by space) in the Address, I fix the very last issues manually in Excel. This is no problem, as I am only working with 44 entries. Speed before unneeded code!

In [3]:
# Saving to Excel:
MetroList.to_excel('/Users/holger/Documents/Python/Harmsen_Repo/Metro-Study/Pickles/Metro Adresses for Final Touches.xlsx')

In [4]:
# Pulling edited version:
MetroList = pd.read_excel('/Users/holger/Documents/Python/Harmsen_Repo/Metro-Study/Pickles/Metro Adresses for Final Touches_DONE.xlsx')

In [5]:
MetroList.head(n=3)

,Unnamed: 0,Address,Railroad,Opening Date,Cityringen
0,0,Godthåbsvej 31 2000 Frederiksberg,Cityringen,2019,1
1,1,Italiensvej 72A 2300 København S,Østamagerbanen,2007,0
2,2,Amagerbro Torv 1 2300 København S,Ørestadsbanen,2002,0


## 3.3) Finding Coordinates to each Metro

In [11]:
MetroList['Coordinates'] = [Geocode_Google(i) for i in MetroList['Address']]

In [12]:
MetroList.to_pickle('/Users/holger/Documents/Python/Harmsen_Repo/Metro-Study/Pickles/Metrolist With Coordinates.pkl')

In [2]:
MetroList = pickle.load(open('/Users/holger/Documents/Python/Harmsen_Repo/Metro-Study/Pickles/Metrolist With Coordinates.pkl','rb'))

# 4) Finding the closest metro

With c. 40 metros and 45,000 apartments, one would have to find metro-distance 40*45,000 times and find each apartment's shortest route. This would be very time-consuming. Instead, the coordinates to each location is found and the closest metro to each apartment is found from the distance between their coordinates. The two are saved as a tuple.

In [81]:
def Closest_Metro(from_num,to_num):
    # Creating a list of the closest metro to each apartment with the same index:
    Closest_Metro = []
    # Looping through each apartment:
    for i in Apartments['Coordinates'][from_num:to_num]:
        # Initiating lowest distance and its metro
        lowest_dist = 1000
        # Looping through metros, finding all distances and substituting the shortest one
        for j in range(0,len(MetroList['Coordinates'])):
            dist = geopy.distance.distance(i, MetroList['Coordinates'][j]).km
            if dist < lowest_dist:
                lowest_dist = dist
                metro = MetroList['Address'][j]
                railroad = MetroList['Railroad'][j]
                opening_date = MetroList['Opening Date'][j]
                cityring_indicator = MetroList['Cityringen'][j]


        Closest_Metro.append({'Distance':lowest_dist,'Metro':metro,'Railroad':railroad,'Opening Date':opening_date,'Cityring':cityring_indicator})
    return Closest_Metro

In [82]:
Apartments['Closest Metro'] = Closest_Metro(0,len(Apartments))

In [83]:
Apartments.to_pickle('/Users/holger/Documents/Python/Harmsen_Repo/Metro-Study/Pickles/Apartment Data 4.pkl')

In [3]:
Apartments = pickle.load(open('/Users/holger/Documents/Python/Harmsen_Repo/Metro-Study/Pickles/Apartment Data 4.pkl','rb'))

In [10]:
Apartments[:3].to_excel('/Users/holger/Documents/Python/Harmsen_Repo/Metro-Study/Other data/illustration of df.xlsx')

In [11]:
Apartments

,Address,Address transformed,Area,Coordinates,Date_sold,Price,Price_mio,Price_sq_m_1000,Rooms,Closest Metro
0,"Stengade 52B, 1. 1, 2200 København N",Stengade 52B 2200 København N,N,"(55.687083, 12.554688)",1994-02-11,300710,0.300710,5.896,1,"{'Distance': 0.6095724783548715, 'Metro': 'Ros..."
1,"Bryggerivej 10, 1. th, 2500 Valby",Bryggerivej 10 2500 Valby,Valby,"(55.660475, 12.518974)",1994-02-11,344101,0.344101,5.550,2,"{'Distance': 0.8403550180969337, 'Metro': 'Car..."
2,"Solvænget 5, 3. tv, 2100 København Ø",Solvænget 5 2100 København Ø,Ø,"(55.719198, 12.57804)",1994-02-11,1090300,1.090300,7.788,4,"{'Distance': 0.9889381894173884, 'Metro': 'Ree..."
3,"Ved Volden 8, 5. tv, 1425 København K",Ved Volden 8 1425 København K,K,"(55.67202, 12.594053)",1994-02-13,415000,0.415000,6.694,2,"{'Distance': 0.18848405881332048, 'Metro': 'To..."
4,"Marstalsgade 38, 2. th, 2100 København Ø",Marstalsgade 38 2100 København Ø,Ø,"(55.703961, 12.589622)",1994-02-14,172000,0.172000,3.071,2,"{'Distance': 0.12255149216648616, 'Metro': 'Øs..."
...,...,...,...,...,...,...,...,...,...,...
56944,"Rådmandsgade 48A, 2. th, 2200 København N",Rådmandsgade 48A 2200 København N,N,"(55.700733, 12.552838)",2020-07-16,816000,0.816000,14.836,2,"{'Distance': 0.23680252000508714, 'Metro': 'Ja..."
56945,"Tycho Brahes Allé 8, 5. tv, 2300 København S",Tycho Brahes Allé 8 2300 København S,S,"(55.6552045, 12.6111575)",2020-07-16,1875000,1.875000,36.765,2,"{'Distance': 0.8496045896388944, 'Metro': 'Øst..."
56946,"Theklavej 49, 2. 79, 2400 København NV",Theklavej 49 2400 København NV,V,"(55.7059296, 12.5251929)",2020-07-16,1550000,1.550000,38.750,1,"{'Distance': 0.9806279241602138, 'Metro': 'Nør..."
56947,"Sundkaj 75, 3. th, 2150 Nordhavn",Sundkaj 75 2150 Nordhavn,NaN,"(55.7100897, 12.5977136)",2020-07-16,3570000,3.570000,51.000,2,"{'Distance': 0.13692980878940642, 'Metro': 'Kl..."
